In [1]:
# needs to be run in python 2
from astropy.io import fits
import numpy as np
from astropy.wcs import WCS
from utils import gauss_kern # from https://github.com/marcoviero/simstack
from utils import circle_mask
from utils import clean_nans
from utils import smooth_psf
from astropy.table import Table
import scipy.signal

from matplotlib import pyplot as plt
from astropy.coordinates import SkyCoord
from astropy import units as units
from astropy.coordinates import match_coordinates_sky

from sklearn.utils import shuffle
import pandas as pd



In [2]:
loc_m = '/Users/Steven/Documents/python_simstack/maps/' # loc of maps

In [3]:
list_names = ['r'] # all the catalogue's you want to run, in this case 1, called r2.csv
map_names = ['SPIRE250_mask4.fits'] # all the maps 
emap_names = ['SPIRE250_err4.fits'] # all the emaps

In [4]:
band = '250' # running on 250um this is needed for PSF
JK = 0 # 4 JK can be run to get another error-bar, in this case its 0 which means
       # no JK, use full map

redshift_nodes  = np.array([0.0, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0,10]) 
# redshift bins, only use when you got redshifts
nuse_all = [0,1,2,3,4,5,6,7,8]

loc = '/Users/Steven/Documents/prepare_simsack/point_mat/'
loc_cat = '/Users/Steven/Documents/python_simstack/cat/'


if band == '250':
  psf = 17.5 #17.6
if band == '350':
  psf = 23.7 #23.9
if band == '500':
  psf = 34.6 #35.2

fwhm = psf


In [5]:
pixsize = 4.0 # in arcsec
len_ax = 15

x, y = np.meshgrid(np.linspace(-1*len_ax*pixsize,len_ax*pixsize,2*len_ax+1), 
                   np.linspace(-1*len_ax*pixsize,len_ax*pixsize,2*len_ax+1))
d = np.sqrt(x*x+y*y)
sigma = fwhm/(2*np.sqrt(2*np.log(2)))
kern = np.exp(-( (d)**2 / ( 2.0 * sigma**2 ) ) )
# the kernel used in simstack


#for nuse in nuse_all:


In [6]:
for q in range(0,np.size(list_names)):
  if map_names[q]== emap_names[q]:
    emap, hds = fits.getdata(loc_m+emap_names[q],2, header = True) # if emap is in same file
    cmap, chd = fits.getdata(loc_m+map_names[q], 1, header = True)
  else:  
    emap, hds = fits.getdata(loc_m+emap_names[q],0, header = True) # if not
    cmap, chd = fits.getdata(loc_m+map_names[q], 0, header = True)
    
    
  tbl = np.array(pd.read_table(loc_cat+list_names[q]+'.csv',sep=','))
  mag = tbl[:,3]
  ra_tot = tbl[:,1]
  dec_tot = tbl[:,2]
  z = tbl[:,5]  
    
  if JK == 1:
    use_JK = (ra_tot > np.mean(ra_tot)) & (dec_tot > np.mean(dec_tot))
  if JK == 2:
    use_JK = (ra_tot > np.mean(ra_tot)) & (dec_tot <= np.mean(dec_tot))
  if JK == 3:
    use_JK = (ra_tot <= np.mean(ra_tot)) & (dec_tot > np.mean(dec_tot))
  if JK == 4:
    use_JK = (ra_tot <= np.mean(ra_tot)) & (dec_tot <= np.mean(dec_tot))
    
  if JK != 0:
    mag = mag[~use_JK]
    ra_tot = ra_tot[~use_JK]
    dec_tot = dec_tot[~use_JK]
    z = z[~use_JK]

    
  radius = 1.1
  zeromask = np.ones(np.shape(cmap))
  ind_map_zero = np.where(clean_nans(cmap) == 0.0) ## in cmap all nan are 0
  zeromask[ind_map_zero] = 0.0
  cms = np.shape(cmap)    
  cw = WCS(chd, naxis = 2) 
  
  '''# removes galaxy pairs within arcsec
  m_tot =mag
  dist_use = 4.0
  c_all = SkyCoord(ra=ra_tot*units.degree, dec=dec_tot*units.degree) 
  table = match_coordinates_sky(c_all, c_all, nthneighbor=2) 
  table2 = np.array(table)
  dist_arc = table2[1,:]*3600
  far = dist_arc > dist_use
  near = dist_arc <= dist_use

  ra_keep = ra_tot[far]
  dec_keep = dec_tot[far]
  m_keep = m_tot[far]

  ra_ap = ra_tot[near]
  dec_ap = dec_tot[near]
  mag_ap = m_tot[near]
  c_ap = c_all[near]

  more = True
  start = 0
  while (more == True) and (np.size(c_ap) != 0) :
    table = match_coordinates_sky(c_ap, c_ap, nthneighbor=2) 
    table2 = np.array(table)
    dist_arc = table2[1,:]*3600
    idx = table2[0,:].astype(int)
    good = (mag_ap < mag_ap[idx]) | (dist_arc > dist_use)

    ra_ap = ra_ap[good]
    dec_ap = dec_ap[good]
    mag_ap = mag_ap[good]
    c_ap = c_ap[good]
    if np.size(ra_ap) == start:
      more = False
    start = np.size(ra_ap)

  ra_tot = np.append(ra_keep, ra_ap)
  dec_tot = np.append(dec_keep, dec_ap)
  mag = np.append(m_keep, mag_ap)  
  '''
  # make magnitude bins  
  if list_names[q] == 'all_1.77_test':
    SOURCE_LIST = np.array([4,8,8.5,9,9.5,10,10.5,11,11.5,20])
  else:
    SOURCE_LIST = np.append(np.linspace(12.4,18,15),np.linspace(18.4,27.6,24))
    while np.max(mag) < SOURCE_LIST[np.size(SOURCE_LIST)-2]:
      SOURCE_LIST = SOURCE_LIST[0:-1]
    SOURCE_LIST[np.size(SOURCE_LIST)-1] = np.max(mag) + 1e-5
    
    
  run = np.size(SOURCE_LIST)
  nlists = run+2 - 1 
  layers=np.zeros([nlists,cms[0],cms[1]])
  ngals_layer = np.zeros(nlists)

  # makes delta function layer's
  for k in range(2,nlists):
      print k, 
      use = (mag >= SOURCE_LIST[k-2]) & (mag < SOURCE_LIST[k-2+1])
      ra = ra_tot[use]
      dec = dec_tot[use]
      if np.size(ra) > 0.5:
        ty,tx = cw.wcs_world2pix(ra, dec, 0)
        # CHECK FOR SOURCES THAT FALL OUTSIDE MAP
        ind_keep = np.where((np.round(tx) >= 0) & (np.round(tx) < cms[0]) & (np.round(ty) >= 0) & (np.round(ty) < cms[1]))
        real_x=np.round(tx[ind_keep]).astype(int)
        real_y=np.round(ty[ind_keep]).astype(int)
        # CHECK FOR SOURCES THAT FALL ON ZEROS
        ind_nz=np.where(cmap[real_x,real_y] != 0 )
      
        nt = np.shape(ind_nz)[1]
        ngals_layer[k] = nt
        print nt, np.size(tx)
        #print 'ngals: ' + str(nt)
        if nt > 0:
          real_x = real_x[ind_nz]
          real_y = real_y[ind_nz]
          for ni in range(nt):
            layers[k, real_x[ni],real_y[ni]]+=1.0

  zA = np.zeros(2)
  DAT = np.array([np.append(zA,SOURCE_LIST[1:]),ngals_layer])
  np.savetxt(loc+'ns_'+list_names[q]+'.cat',DAT.T, delimiter="   ", fmt='%s', newline='\n')

  cfits_flat = np.asarray([])

  # STEP 2  - Convolve Layers and put in pixels`
  flattened_pixmap = np.sum(layers,axis=0)
  total_circles_mask = circle_mask(flattened_pixmap, radius * fwhm, pixsize)
  ind_fit = np.where((total_circles_mask >= 1) & (zeromask != 0))
  del total_circles_mask
  nhits = np.shape(ind_fit)[1]
  cfits_flat = np.asarray([])
    
   
  once = True
  twice = True
  for u in range(nlists):
    print 'nsource', np.sum(layers[u,:,:])
    layer = layers[u,:,:]
    if u > 1.5:
      if np.sum(layer) > 0.5:
        tmap = scipy.signal.convolve2d(layer, kern, mode='same')
      else:
        tmap = layer  
    else:
      if (u == 0) :
        print u
        layer = layer + 1
        tmap = layer 
      elif (u == 1):
        if (np.size(ind_map_zero) > 0):
          print 'nanl'
          layer[ind_map_zero] = 1

        tmap = scipy.signal.convolve2d(layer, kern, mode='same')
    print np.min(tmap[ind_fit]) ,np.max(tmap[ind_fit])  

    print 'psf', np.sum(tmap[ind_fit]), np.sum(tmap), u
    cfits_flat = np.append(cfits_flat,np.ndarray.flatten(tmap[ind_fit]))



  DAT = np.array([cmap[ind_fit],emap[ind_fit]]).T
  print np.shape(DAT)
  t = Table(DAT, names=('a','b'))

  t.write(loc+'p_map_'+band+'_'+list_names[q]+'_FWHM'+str(psf)+'.fits', format='fits')

  t = Table([cfits_flat], names=('a'))
  t.write(loc+'pm_'+band+'_beth_'+list_names[q]+'_FWHM'+str(psf)+'.fits', format='fits')



## now all pointing matrices have been created
## next box is simple_stacking_bot.py

2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 2 2
17 38 38
18 107 109
19 212 216
20 502 506
21 862 875
22 1345 1363
23 1938 1970
24 2844 2884
25 3973 4028
26 5713 5793
27 8082 8207
28 12190 12386
29 17893 18192
30 26537 27006
31 38249 38981
32 53367 54267
33 70125 71336
34 89246 90693
35 100531 102077
36 87527 88707
37 30310 30556
38 101 101
nsource 0.0
0
1.0 1.0
psf 1418537.0 4459319.0 0
nsource 0.0
nanl
0.0 20.123080123374024
psf 1461888.7886128365 65813757.28733286 1
nsource 0.0
0.0 0.0
psf 0.0 0.0 2
nsource 0.0
0.0 0.0
psf 0.0 0.0 3
nsource 0.0
0.0 0.0
psf 0.0 0.0 4
nsource 0.0
0.0 0.0
psf 0.0 0.0 5
nsource 0.0
0.0 0.0
psf 0.0 0.0 6
nsource 0.0
0.0 0.0
psf 0.0 0.0 7
nsource 0.0
0.0 0.0
psf 0.0 0.0 8
nsource 0.0
0.0 0.0
psf 0.0 0.0 9
nsource 0.0
0.0 0.0
psf 0.0 0.0 10
nsource 0.0
0.0 0.0
psf 0.0 0.0 11
nsource 0.0
0.0 0.0
psf 0.0 0.0 12
nsource 0.0
0.0 0.0
psf 0.0 0.0 13
nsource 0.0
0.0 0.0
psf 0.0 0.0 14
nsource 0.0
0.0 0.0
psf 0.0 0.0 15
nsource 2.0
0.0 1.0
psf 39.55024427094518 43.3761029

In [9]:
import numpy as np
from utils import clean_args
from utils import clean_nans
from lmfit import Parameters, minimize #, fit_report
from astropy.io import fits
import time


t0 = time.time()

def simultaneous_stack_array_oned(p, layers_1d, data1d, err1d = None, arg_order = None):
  ''' Function to Minimize written specifically for lmfit '''
  v = p.valuesdict()
  len_model = len(data1d)
  nlayers = len(layers_1d)/len_model
  model = np.zeros(len_model)
  for i in range(nlayers):
  	model[:] += layers_1d[i*len_model:(i+1)*len_model] * v[v.keys()[i]]
 
  #model -= np.mean(model) ###removed by me!

  #return (data1d - model)/err1d
  return ( (data1d - model)/err1d )


loc = '/Users/Steven/Documents/prepare_simsack/point_mat/'
#


back = 0 # -1*np.mean(imap)




SOURCE_LIST = np.append(np.linspace(12.4,18,15),np.linspace(18.4,26.4,21))
run = np.size(SOURCE_LIST)-1

header = 'Xi'
for q in range(run):
    header = header + ' TF'+str(SOURCE_LIST[q+1])

boot = 10
TF = np.zeros([boot,run+2])
chi = np.zeros(boot)+1e9
total = np.zeros(10)



redshift_nodes  = np.array([0.01, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0,10]) 

  
names = ['r']

wave = '250'
for q in range(0,np.size(names)):
#for q in range(3,4):

    name2 = names[q]
    name = 'p_map_'+wave+'_'+name2+'.fits'
    hdu = fits.open(loc+name)
    img = hdu[1].data
    imap = np.ndarray.flatten(img['a'])
    ierr = np.ndarray.flatten(img['b'])


    imap = imap - np.mean(imap)
    name = 'pm_'+wave+'_beth_'+name2+'2.fits'
    hdu = fits.open(loc+name)
    img = hdu[1].data
    cfits_flat = np.ndarray.flatten(img['a'])


    len_model = len(imap)

    print 'helo'
    t2 = time.time()
    nlayers = len(cfits_flat)/len_model 
    chi_best = 1e9

    cfits_flat_use = np.zeros(np.size(cfits_flat))

    bs_sample = np.random.randint(0, len_model,len_model)

    fit_params = Parameters()
    for iarg in range(nlayers):
      arg = 'name'+str(iarg)+'good'
      fit_params.add(arg,value= 1e-6*np.random.randn())
      ## coment out
      use = cfits_flat[iarg*len_model:(iarg+1)*len_model]
      cfits_flat_use[iarg*len_model:(iarg+1)*len_model] = use[bs_sample]

    ## only for boothstrap
    #cov_ss_1d = minimize(simultaneous_stack_array_oned, fit_params,
    #	  args=(cfits_flat_use,), kws={'data1d':imap[bs_sample],'err1d':ierr[bs_sample]}, nan_policy = 'propagate')

    cov_ss_1d = minimize(simultaneous_stack_array_oned, fit_params,
          args=(cfits_flat,), kws={'data1d':imap,'err1d':ierr}, nan_policy = 'propagate')#, xtol = 1e-15, ftol = 1e-15, epsfcn = 0.001) 


    values = np.array(cov_ss_1d.params)
    #TF[q,:] = values
    print(values)
    model = np.zeros(len_model)
    for i in range(nlayers):
        model[:] += cfits_flat[i*len_model:(i+1)*len_model] * values[i]



    t1 = time.time()
    tpass = t1-t2	
    print "Total time run                       :",tpass/60, '  minuts'
    
    DAT =  np.array(values).T
    np.savetxt(loc+'res_'+wave+'_'+name2+'tt.cat', DAT, delimiter="   ", fmt='%s', newline='\n')

helo
[-7.09632071e-03  4.09908332e-04  3.66799081e-07 -1.40333955e-07
  9.17462581e-08 -1.85534700e-06 -1.87619734e-07 -2.20781608e-07
  6.46218808e-07 -6.33203254e-07 -3.35221396e-07  1.40236869e-07
 -2.42123939e-07  1.05761502e-06  1.49376891e-06 -8.03781588e-07
  6.74310203e-02  3.50219562e-02  2.16768295e-02  1.49157465e-02
  1.10643344e-02  8.21384474e-03  6.59408568e-03  5.03316039e-03
  4.19710475e-03  3.65598456e-03  3.32352272e-03  2.69713272e-03
  2.40714841e-03  2.00943857e-03  1.63737247e-03  1.22875260e-03
  9.31265987e-04  6.28062659e-04  3.94086671e-04  3.27961250e-04
  2.68496520e-04  2.20883604e-04]
Total time run                       : 1.90590998332   minuts
